In [336]:
%matplotlib qt5
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform
import scipy

#### Skriv i master at jeg kan ha multidomener, men har ikke tatt hensyn til dette i prosesseringen. 

In [337]:
def transform_to_ASI(data, corners, shape=(512,512)):
    src_points = np.array([
        [corners[0][0] + 2, corners[0][1] + 2],
        [corners[1][0] - 2, corners[1][1] + 2],
        [corners[2][0] + 2, corners[2][1] - 2],
        [corners[3][0] - 2, corners[3][1] - 2]
    ])

    dst_points = np.array([
        [  0,   0],
        [512,   0],
        [  0, 512],
        [512, 512]
    ])

    tform = transform.ProjectiveTransform()
    tform.estimate(src_points, dst_points)
    transformed_image = transform.warp(data, tform.inverse, output_shape=shape)

    return transformed_image
    

In [338]:
def rgb_to_hsv_pixel(r, g, b):

    maxc = max(r, g, b)
    minc = min(r, g, b)
    v = maxc
    s = (maxc - minc) / maxc if maxc != 0 else 0

    if maxc == minc:
        h = 0
    elif maxc == r:
        h = (60 * ((g - b) / (maxc - minc)) + 360) % 360
    elif maxc == g:
        h = (60 * ((b - r) / (maxc - minc)) + 120) % 360
    elif maxc == b:
        h = (60 * ((r - g) / (maxc - minc)) + 240) % 360
    
    return h, s, v

In [339]:
def rgb_to_hsv_image(rgb):
    
    h_channel = np.zeros((rgb.shape[0], rgb.shape[1]))
    s_channel = np.zeros((rgb.shape[0], rgb.shape[1]))
    v_channel = np.zeros((rgb.shape[0], rgb.shape[1]))

    for i in range(rgb.shape[0]):
        for j in range(rgb.shape[1]):
            r, g, b = rgb[i, j]
            h, s, v = rgb_to_hsv_pixel(r, g, b)
            h_channel[i, j] = h
            s_channel[i, j] = s
            v_channel[i, j] = v

    hsv = np.stack((h_channel, s_channel, v_channel), axis=-1)
    
    return hsv

In [340]:
def group_islands(transformed_image):
    black_pixel_threshold = 0.01  # Define a threshold for considering a pixel as black
    mask = np.all(transformed_image < black_pixel_threshold, axis=-1)

    # Invert the mask to get non-black pixels as 1
    mask = ~mask

    # Step 4: Label the connected components 
    # labeled_array is the "wrong way". that is, y, x = labeled_array
    labeled_array, num_features = scipy.ndimage.label(mask)

    # Step 5: Calculate the centroids of each island
    centroids = scipy.ndimage.center_of_mass(mask, labeled_array, range(1, num_features + 1))
    return labeled_array, centroids, num_features

In [341]:
def remove_outer_layers(labeled_islands, num_islands, layers=2):
    # Create a mask for all islands
    mask = labeled_islands > 0

    # Perform binary erosion twice to remove two layers
    eroded_mask = mask.copy()
    for _ in range(layers):
        eroded_mask = scipy.ndimage.binary_erosion(eroded_mask)

    # Create a new labeled array for the eroded islands
    eroded_labeled_islands, new_num_islands = scipy.ndimage.label(eroded_mask)

    # Map the original labels to the new eroded labels
    new_labeled_islands = np.zeros_like(labeled_islands)
    for original_label in range(1, num_islands + 1):
        original_island_mask = labeled_islands == original_label
        eroded_island_mask = np.logical_and(original_island_mask, eroded_mask)
        if np.any(eroded_island_mask):
            new_label = eroded_labeled_islands[eroded_island_mask][0]
            new_labeled_islands[eroded_island_mask] = new_label

    return new_labeled_islands, new_num_islands

In [342]:
def find_average_magnetisation(rgb, labeled_islands, num_islands):
    # List to store the average hue of each island
    average_magnetisation = []

    # Iterate through each island
    for label in range(1, num_islands + 1):
        # Mask for the current island
        island_mask = labeled_islands == label

        # Extract the RGB values for the current island
        island_pixels_rgb = rgb[island_mask]

        # Convert the RGB values to HSV values
        island_pixels_hsv = np.array([rgb_to_hsv_pixel(rgb[0], rgb[1], rgb[2]) for rgb in island_pixels_rgb])

        # Extract the H (hue) values from the HSV values
        h_values = island_pixels_hsv[:, 0]

        # Calculate the average hue for the current island
        h_average = average_hsv_pixels(island_pixels_hsv)

        # Store the average hue
        average_magnetisation.append(h_average)

    return average_magnetisation



In [343]:
def horizontal_or_vertical_array(num_islands, width):
    # width is the number of horizontal islands, length is the number of vertical islands
    pattern = np.array([1]*width + [0]*(width+1))

    # Repeat the pattern to achieve the desired length
    repeated_pattern = np.tile(pattern, (num_islands // len(pattern)) + 1)[:num_islands]

    return repeated_pattern

In [344]:
def make_quaternary_magnetisation(average_magnetisation, width):
    quaternary_values = []
    # Must change this to match the actual value
    horizontal = horizontal_or_vertical_array(len(average_magnetisation), width)
    for i in range(len(average_magnetisation)):

        # Determine the quaternary value based on the given ranges
        if (250 <= average_magnetisation[i] or average_magnetisation[i] < 70) and horizontal[i] == 1:
            quaternary_values.append(0)
        elif (340 <= average_magnetisation[i] or average_magnetisation[i] < 160) and horizontal[i] == 0:
            quaternary_values.append(1)
        elif (70 <= average_magnetisation[i] < 250) and horizontal[i] == 1:
            quaternary_values.append(2)
        elif (160 <= average_magnetisation[i] < 340) and horizontal[i] == 0:
            quaternary_values.append(3)
    
    return quaternary_values

In [345]:
def draw_arrows(centre_positions, magnetisation_directions, filename):
    # Create a new plot
    fig, ax = plt.subplots()

    # Define the arrow properties with larger and wider arrows
    arrow_props = {
        0: {'dx': -10, 'dy': 0, 'color': 'red'},         # Pointing right
        1: {'dx': 0, 'dy': 10, 'color': 'green'},         # Pointing up
        2: {'dx': 10, 'dy': 0, 'color': 'turquoise'},      # Pointing left
        3: {'dx': 0, 'dy': -10, 'color': 'purple'}        # Pointing down
    }

    # Define head width and length
    head_width = 6
    head_length = 6


    # Iterate over the positions and corresponding magnetisation directions
    for (y, x), direction in zip(centre_positions, magnetisation_directions):
        # Get the arrow properties based on the direction
        props = arrow_props[direction]

        # Calculate the total length of the arrow (body + head)
        total_length_x = props['dx'] + (head_length if props['dx'] != 0 else 0)
        total_length_y = props['dy'] + (head_length if props['dy'] != 0 else 0)

        # Calculate the starting point so the entire arrow is centered
        start_x = x - total_length_x / 2 
        start_y = y - total_length_y / 2
        if direction == 0:
            start_x = start_x
        elif direction == 1:
            start_y = start_y
            start_x -= head_length
        elif direction == 2:
            start_x -= head_length
        elif direction == 3:
            start_y += head_length
            start_x -= head_length

    
        # Draw the arrow with larger head width, head length, and shaft width
        ax.arrow(start_x, start_y, props['dx'], props['dy'], head_width=head_width, head_length=head_length, width=2, fc=props['color'], ec=props['color'])


    # Set the aspect of the plot to be equal
    ax.set_aspect('equal')

    ax.set_xticks([])
    ax.set_yticks([])

    #fig.savefig(f"{filename}.pdf", bbox_inches='tight')

    # Display the plot
    plt.show()

In [346]:
def average_hsv_pixels(hsv_pixels):
    """
    Calculate the average of an array of HSV pixels.
    
    Args:
    hsv_pixels (list of tuples or np.ndarray): Array of HSV pixels (each pixel is a tuple of hue in degrees, saturation, value).
    
    Returns:
    tuple: The average HSV pixel.
    """
    hsv_pixels = np.array(hsv_pixels)
    
    # Separate the Hue, Saturation, and Value components
    hues = hsv_pixels[:, 0]

    # Convert Hue from degrees to radians
    hues_rad = np.deg2rad(hues)
    
    # Convert Hue to unit vectors
    x_components = np.cos(hues_rad)
    y_components = np.sin(hues_rad)
    
    # Sum the unit vectors
    x_avg = np.sum(x_components) / len(hsv_pixels)
    y_avg = np.sum(y_components) / len(hsv_pixels)
    
    # Calculate the average Hue from the unit vectors
    avg_h_rad = np.arctan2(y_avg, x_avg)
    avg_h_deg = np.rad2deg(avg_h_rad) % 360
    
    return avg_h_deg


In [347]:
with open('corners_data/015_2c6nm_0rot_512x512_4length_30min_1c4_corners.npy', 'rb') as f:
    corners = np.load(f)
with open('images_data/015_2c6nm_0rot_512x512_4length_30min_1c4.npy', 'rb') as f:
    rgb = np.load(f)
transformed_image = transform_to_ASI(rgb, corners, shape=(512, 512))
fig, ax = plt.subplots()
ax.imshow(np.flipud(transformed_image))

In [348]:
labeled_islands, centres_islands, num_islands = group_islands(transformed_image)
labeled_islands, num_islands = remove_outer_layers(labeled_islands, num_islands, layers=3)

In [349]:
"""with open('015_2c6nm_0rot_512x512_4length_30min_1c4_magnet_centres.npy', 'wb') as f:
    np.save(f, centres_islands)"""

"with open('015_2c6nm_0rot_512x512_4length_30min_1c4_magnet_centres.npy', 'wb') as f:\n    np.save(f, centres_islands)"

In [350]:
avg_mag = find_average_magnetisation(transformed_image, labeled_islands, num_islands)
quartenary = make_quaternary_magnetisation(avg_mag, 13)
draw_arrows(centres_islands, quartenary, '015_2c6nm_0rot_512x512_4length_30min_1c4')

In [351]:
"""with open('015_2c6nm_0rot_512x512_4length_30min_1c4_quartenary_magnetisation.npy', 'wb') as f:
    np.save(f, quartenary)"""

"with open('015_2c6nm_0rot_512x512_4length_30min_1c4_quartenary_magnetisation.npy', 'wb') as f:\n    np.save(f, quartenary)"

In [352]:
"""def plot_color_wheel():
    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    
    hues = np.linspace(0, 360, 360)
    for h in hues:
        ax.bar(np.deg2rad(h), 1, color=plt.cm.hsv(h / 360), edgecolor='none')
    
    ax.set_yticklabels([])
    ax.set_xticks(np.deg2rad(np.arange(0, 360, 30)))
    ax.set_xticklabels([f'{int(angle)}°' for angle in np.arange(0, 360, 30)])
    
    plt.show()

# Call the function to plot the color wheel
plot_color_wheel()"""

"def plot_color_wheel():\n    fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))\n    ax.set_theta_offset(np.pi / 2)\n    ax.set_theta_direction(-1)\n    \n    hues = np.linspace(0, 360, 360)\n    for h in hues:\n        ax.bar(np.deg2rad(h), 1, color=plt.cm.hsv(h / 360), edgecolor='none')\n    \n    ax.set_yticklabels([])\n    ax.set_xticks(np.deg2rad(np.arange(0, 360, 30)))\n    ax.set_xticklabels([f'{int(angle)}°' for angle in np.arange(0, 360, 30)])\n    \n    plt.show()\n\n# Call the function to plot the color wheel\nplot_color_wheel()"